In [1]:
%load_ext autoreload
%autoreload 0

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, f1_score, precision_score, recall_score, log_loss

In [3]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

In [4]:
feature_descriptions = {
    "Sex": "Gender of passenger",
    "Deck": "The deck the passenger had their cabin on",
    "PassengerClass": "The class of the ticket: 1st, 2nd or 3rd class",
    "Fare": "The amount of money people paid", 
    "No_of_relatives_on_board": "number of siblings, spouses, parents plus children on board",
    "Embarked": "the port where the passenger boarded the Titanic. Either Southampton, Cherbourg or Queenstown",
    "Age": "Age of the passenger",
    "No_of_siblings_plus_spouses_on_board": "The sum of the number of siblings plus the number of spouses on board",
    "No_of_parents_plus_children_on_board" : "The sum of the number of parents plus the number of children on board",
}

# ClassifierBunch example

## Load classifier data:
    - predicting probability that a person on the titanic survived

In [5]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

## RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestClassifierBunch(model, X_test, y_test, metric=roc_auc_score, 
                               shap='tree',
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               descriptions=feature_descriptions,
                               labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        decision_trees=True)
db.run(8052)

## LogisticRegression

In [25]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

explainer = ClassifierBunch(model, X_test, y_test, metric=roc_auc_score, 
                               shap='linear',
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               descriptions=feature_descriptions,
                               labels=['Not survived', 'Survived'])

In [26]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False, # Linear models have no interactions
                        decision_trees=False) # Linear models have no decision trees
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating pred_percentiles...
Calculating prediction probabilities...
Calculating importances...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [28/May/2020 15:45:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 15:45:24] "POST /_dash-update-component

## XGBClassifier

In [7]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

explainer = ClassifierBunch(model, X_test, y_test, roc_auc_score, shap='tree',
                                   cats=['Sex', 'Deck', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) #only model_output = "raw" supportted with interaction values for now
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating pred_percentiles...
Calculating prediction probabilities...
Calculating importances...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2020 18:07:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:12] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 18:07:13] "POST /_dash-update-component

## LGBMClassifier

In [17]:
from lightgbm.sklearn import LGBMClassifier

model = LGBMClassifier()
model.fit(X_train, y_train)

explainer = ClassifierBunch(model, X_test, y_test, roc_auc_score, shap='tree',
                                   cats=['Sex', 'Deck', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True) 
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating pred_percentiles...
Calculating prediction probabilities...
Calculating importances...
Calculating shap interaction values...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2020 16:53:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:53:46] "POST /_dash-update-component

## CatBoostClassifier

In [6]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100, learning_rate=100)



In [ ]:
model.fit(X_train, y_train)

In [ ]:

explainer = ClassifierBunch(model, X_test, y_test, roc_auc_score, shap='tree',
                                   cats=['Sex', 'Deck', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) 
db.run(8052)

# RegressionBunch example

## Load regression data:
    - predicting the fare that a titanic passenger paid for their ticket

In [6]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

## RandomForestRegressor

In [22]:
model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestRegressionBunch(model, X_test, y_test, r2_score,
                shap='tree', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        decision_trees=True)
db.run(8052)

## LinearRegression

In [23]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='linear', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [20]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        §contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction percentiles...
Calculating shap values...
Calculating importances...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [28/May/2020 12:08:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2020 12:08:10] "POST /_dash-update-component

## XGBRegressor

In [14]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='linear', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) #only model_output = "raw" supportted with interaction values for now
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...


KeyError: 'weight'

## LightGBM

In [9]:
from lightgbm.sklearn import LGBMRegressor

model = LGBMRegressor()
model.fit(X_train, y_train)

explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='tree', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True) 
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction percentiles...
Calculating shap values...
Calculating importances...
Calculating shap interaction values...
Calculating shap interaction values...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2020 16:37:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:37:38] "POST /_dash-update-component

## CatBoost

In [12]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=100, learning_rate=0.1, verbose=0)
model.fit(X_train, y_train)

explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='tree', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) # interactions not supported for now due to this issue: https://github.com/slundberg/shap/issues/480 
db.run(8052)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction percentiles...
Calculating shap values...
Calculating importances...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2020 16:45:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2020 16:45:41] "POST /_dash-update-component